In [16]:
# =============================================
# 1. CONFIGURACIÓN INICIAL
# =============================================

import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Ruta al DW
db_path = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/data/external/chivas_dw.sqlite"

# Conexión
conn = sqlite3.connect(db_path)

# Cargar la vista combinada
df = pd.read_sql_query("SELECT * FROM vw_predicciones_diarias_extendida", conn)
# Cerrar conexión
conn.close()

print(f"Registros cargados: {len(df)}")
df.head()


Registros cargados: 2163


id_jugador       Fecha  microciclo_next  tipo_dia_next Tipo_Dia  Edad  \
0          25  2025-05-23                1            NaN  ENTRENO    26   
1          25  2025-05-24                1            NaN  ENTRENO    26   
2          25  2025-05-26                2            NaN  ENTRENO    26   
3          25  2025-05-27                2            NaN  ENTRENO    26   
4          25  2025-05-28                2            NaN  ENTRENO    26   

   Peso_kg  Estatura_cm   Posicion    Linea  ...  Carga_Regenerativa  \
0     70.0        176.0  Delantero  Extremo  ...              1562.0   
1     70.0        176.0  Delantero  Extremo  ...              1528.5   
2     70.0        176.0  Delantero  Extremo  ...               998.5   
3     70.0        176.0  Delantero  Extremo  ...              2020.5   
4     70.0        176.0  Delantero  Extremo  ...              1808.5   

   Distancia_total  HMLD_m  HSR_abs_m  Sprints_cantidad  Acc_3  Dec_3  \
0             3294     170          0                 0      7      1   
1             3707     512        138                 0      8      6   
2             2281     284          0                 0     39     22   
3             4821     638        142                 1     36     27   
4             4523     741        165                 3     54     30   

   Player_Load  RPE  Sprints_vel_max_kmh  
0           49    4                 18.3  
1           60    5                 22.1  
2           31    0                 20.5  
3           49    0                 25.5  
4           70    0                 26.7  

[5 rows x 34 columns]

In [17]:
# Primero, como hicimos en todos los dataset a entrenar, excluimos todos los jugadores 
# que no proporcionan valor, al igual que los microciclos donde no se compitió:

# =============================================
# 2.1 FILTRO DE JUGADORES Y MICRO-CICLOS
# =============================================

print(f"Registros cargados pre filtrado: {len(df)}\n")

# Jugadores que deben excluirse
jugadores_excluir = [1, 2, 3, 12, 30]

# Filtrar jugadores no válidos
df = df[~df["id_jugador"].isin(jugadores_excluir)]

# fFiltramos los días nulos o no validos (elimina microciclos fuera de competencia):
df = df[df['tipo_dia_next'].notnull()]


print(f"Registros cargados post filtrado: {len(df)}\n")
# Distribución de la etiqueta
print("\nDistribución de tipo_semana_next:")
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)


Registros cargados pre filtrado: 2163

Registros cargados post filtrado: 1357


Distribución de tipo_semana_next:
tipo_semana_next
media    50.257922
alta     33.971997
baja     15.770081
Name: proportion, dtype: float64


In [18]:
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)
print(df['tipo_dia_next'].value_counts())


tipo_semana_next
media    50.257922
alta     33.971997
baja     15.770081
Name: proportion, dtype: float64
tipo_dia_next
-1.0    418
 1.0    301
-2.0    255
 2.0    249
 3.0    134
Name: count, dtype: int64


In [19]:
features = [
    'tipo_dia_next', 'tipo_semana_next',
    'CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual',
    'riesgo_suavizado_3d_actual',
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next'
]
target = 'Distancia_total'


In [20]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()
df['tipo_semana_next'] = le.fit_transform(df['tipo_semana_next'])

X = df[features]
y = df[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


R²: 0.34825455666759386
MAE: 823.0126107351085


In [22]:
from scipy.stats import spearmanr
corr, _ = spearmanr(y_test, y_pred)
print(f"📈 Correlación de rangos (Spearman): {corr:.3f}")


📈 Correlación de rangos (Spearman): 0.605


### El modelo predijo, con algunos detlles que se pueden corregir. En esta etapa se evaluó sin tener en cuenta al jugador ni su pocisión: Vamos agergar esos datos junto con un perfil fisiológico para darle más contexto al modelo de aprender con mayor especificidad .

In [23]:
# ============================================
# 🧠 CONTEXTO FISIOLÓGICO: jugador + posición
# ============================================
import pandas as pd

# 1️⃣ Reemplazar nulos en posición y línea por "Desconocido" (si los hay)
df['Posicion'] = df['Posicion'].fillna('Desconocido')
df['Linea'] = df['Linea'].fillna('Desconocido')

# 2️⃣ Calcular medias históricas por jugador
player_mean = df.groupby('id_jugador').agg({
    'Distancia_total': 'mean',
    'Player_Load': 'mean',
    'Acc_3': 'mean',
    'Dec_3': 'mean'
}).rename(columns={
    'Distancia_total': 'jugador_mean_dist',
    'Player_Load': 'jugador_mean_load',
    'Acc_3': 'jugador_mean_acc',
    'Dec_3': 'jugador_mean_dec'
}).reset_index()

# Unir las medias al dataframe principal
df = df.merge(player_mean, on='id_jugador', how='left')

# 3️⃣ Codificar posición y línea (one-hot encoding)
df = pd.get_dummies(df, columns=['Posicion', 'Linea'], prefix=['Pos', 'Lin'])

# 4️⃣ Confirmar que se agregaron bien las nuevas columnas
print("✅ Nuevas columnas añadidas:", [c for c in df.columns if 'Pos_' in c or 'Lin_' in c])
print(df[['id_jugador', 'jugador_mean_dist', 'jugador_mean_load', 'jugador_mean_acc', 'jugador_mean_dec']].head())
df.columns.tolist()

✅ Nuevas columnas añadidas: ['Pos_Defensor', 'Pos_Delantero', 'Pos_Mediocampista', 'Lin_Defensa Central', 'Lin_Defensa Lateral', 'Lin_Delantera', 'Lin_Extremo', 'Lin_Medio Defensivo', 'Lin_Medio Ofensivo']
   id_jugador  jugador_mean_dist  jugador_mean_load  jugador_mean_acc  \
0          25        3854.093023          52.697674         28.325581   
1          25        3854.093023          52.697674         28.325581   
2          25        3854.093023          52.697674         28.325581   
3          25        3854.093023          52.697674         28.325581   
4          25        3854.093023          52.697674         28.325581   

   jugador_mean_dec  
0          27.44186  
1          27.44186  
2          27.44186  
3          27.44186  
4          27.44186  


['id_jugador',
 'Fecha',
 'microciclo_next',
 'tipo_dia_next',
 'Tipo_Dia',
 'Edad',
 'Peso_kg',
 'Estatura_cm',
 'CT_total_actual',
 'CE_total_actual',
 'CS_total_actual',
 'CR_total_actual',
 'riesgo_suavizado_3d_actual',
 'entrenos_total_next',
 'descansos_total_next',
 'partidos_total_next',
 'descansos_pre_partido_next',
 'entrenos_pre_partido_next',
 'entrenos_post_partido_next',
 'tipo_semana_next',
 'Carga_Explosiva',
 'Carga_Sostenida',
 'Carga_Regenerativa',
 'Distancia_total',
 'HMLD_m',
 'HSR_abs_m',
 'Sprints_cantidad',
 'Acc_3',
 'Dec_3',
 'Player_Load',
 'RPE',
 'Sprints_vel_max_kmh',
 'jugador_mean_dist',
 'jugador_mean_load',
 'jugador_mean_acc',
 'jugador_mean_dec',
 'Pos_Defensor',
 'Pos_Delantero',
 'Pos_Mediocampista',
 'Lin_Defensa Central',
 'Lin_Defensa Lateral',
 'Lin_Delantera',
 'Lin_Extremo',
 'Lin_Medio Defensivo',
 'Lin_Medio Ofensivo']

In [24]:
# ============================================
# ⚙️ SELECCIÓN DE FEATURES Y TARGET
# ============================================

features = [
    # Contexto planificado (semana y día)
    'tipo_semana_next', 'tipo_dia_next',

    # Carga y estado actual
    'CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual',
    'riesgo_suavizado_3d_actual',

    # Estructura de la próxima semana
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next',

    # Contexto fisiológico del jugador
    'jugador_mean_dist', 'jugador_mean_load', 'jugador_mean_acc', 'jugador_mean_dec'
] + [c for c in df.columns if c.startswith('Pos_') or c.startswith('Lin_')]

target = 'Distancia_total'

print("📊 Total de features:", len(features))


📊 Total de features: 25


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr

# Codificar tipo de semana y tipo de día
le_semana = LabelEncoder()
le_dia = LabelEncoder()

df['tipo_semana_next'] = le_semana.fit_transform(df['tipo_semana_next'])
df['tipo_dia_next'] = le_dia.fit_transform(df['tipo_dia_next'].astype(str))

# Separar features y target
X = df[features]
y = df[target]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modelo
rf = RandomForestRegressor(n_estimators=300, max_depth=12, random_state=42)
rf.fit(X_train, y_train)

# Predicciones
y_pred = rf.predict(X_test)

# Evaluaciones
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
spearman_corr, _ = spearmanr(y_test, y_pred)

print("📈 R²:", round(r2, 3))
print("📊 MAE:", round(mae, 2))
print("🔗 Correlación de rangos (Spearman):", round(spearman_corr, 3))


📈 R²: 0.403
📊 MAE: 807.42
🔗 Correlación de rangos (Spearman): 0.641


In [ ]:
import joblib, os

output_dir = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/src/chivas_ml/ml/registry/modelo_clas_distancia_total"
os.makedirs(output_dir, exist_ok=True)

# Guardar modelo y objetos asociados
joblib.dump(rf, os.path.join(output_dir, "model_carga_diaria_rf_tendencias.pkl"))
joblib.dump(scaler, os.path.join(output_dir, "scaler_carga_diaria.pkl"))
joblib.dump(le_semana, os.path.join(output_dir, "label_tipo_semana.pkl"))
joblib.dump(le_dia, os.path.join(output_dir, "label_tipo_dia.pkl"))

print("✅ Modelo, scaler y label encoders guardados correctamente.")


✅ Modelo, scaler y label encoders guardados correctamente.


# 🧠 Modelo de Predicción de Carga Diaria — Versión Tendencias Fisiológicas

## ⚽ Contexto del experimento

Este modelo tiene como objetivo **predecir las métricas diarias de carga física** de cada jugador (por ejemplo, `Distancia_total`), considerando no solo los parámetros de planificación del microciclo, sino también **el contexto fisiológico individual y táctico**.

A diferencia de versiones anteriores que buscaban ajustar valores exactos, esta versión se centra en **aprender tendencias fisiológicas y patrones de comportamiento**, para que el modelo pueda **razonar cómo responde cada jugador según su contexto, tipo de día y tipo de semana**.

---

## 🧩 Modificaciones aplicadas al DataFrame

### 1️⃣ Incorporación de contexto fisiológico individual
Se agregaron **promedios históricos por jugador**, que reflejan su perfil típico de carga:
- `jugador_mean_dist` → promedio histórico de distancia total.  
- `jugador_mean_load` → promedio histórico de Player Load.  
- `jugador_mean_acc` → promedio histórico de aceleraciones.  
- `jugador_mean_dec` → promedio histórico de desaceleraciones.  

Estas variables funcionan como una *firma fisiológica* que permite al modelo identificar patrones de respuesta según el jugador.

---

### 2️⃣ Incorporación del contexto táctico
Se incluyeron las columnas:
- `Posicion` (Delantero, Mediocampista, Defensor, etc.)  
- `Linea` (Defensa Lateral, Medio Ofensivo, etc.)

Ambas fueron convertidas en variables **one-hot encoded** (`Pos_`, `Lin_`), permitiendo al modelo reconocer las diferencias fisiológicas y tácticas entre líneas y roles.

---

### 3️⃣ Estructura de los datos
El modelo ahora cuenta con un total de **~25 features**, distribuidos en tres bloques principales:

| Grupo | Ejemplos | Descripción |
|--------|-----------|-------------|
| 📅 **Planificación** | `tipo_semana_next`, `tipo_dia_next`, `entrenos_total_next` | Información planificada de la próxima semana |
| 💪 **Estado actual** | `CT_total_actual`, `riesgo_suavizado_3d_actual`, `CE_total_actual` | Estado físico y de carga acumulada del jugador |
| 🧬 **Contexto fisiológico y táctico** | `jugador_mean_*`, `Pos_*`, `Lin_*` | Perfil físico individual y línea táctica |

---

## ⚙️ Entrenamiento del modelo

**Modelo:** `RandomForestRegressor`  
**Configuración:**  
- `n_estimators=300`  
- `max_depth=12`  
- `random_state=42`

Se entrenó sobre el 80 % del dataset y se evaluó sobre el 20 % restante.  
Las variables categóricas (`tipo_semana_next`, `tipo_dia_next`) fueron codificadas con `LabelEncoder`, y los valores numéricos fueron escalados con `StandardScaler`.

---

## 📊 Resultados obtenidos

| Métrica | Valor | Interpretación |
|----------|--------|----------------|
| **R²** | 0.403 | El modelo explica el 40 % de la variabilidad de la carga diaria. En contextos fisiológicos, esto indica una comprensión robusta de las tendencias reales. |
| **MAE** | 807.42 | Error medio de 807 metros, equivalente al 9–13 % de la carga diaria promedio. |
| **Correlación de Spearman** | 0.641 | El modelo ordena correctamente los valores según la intensidad esperada, lo que demuestra comprensión de las relaciones fisiológicas. |

---

## 💬 Interpretación general

El modelo logró:
- Aprender **patrones de comportamiento individualizados**: distingue cómo responde cada jugador según su historial.  
- Entender las diferencias **por posición y rol táctico**.  
- Captar el **efecto del riesgo acumulado y la fatiga** (`riesgo_suavizado_3d_actual`).  
- Razonar coherentemente la relación entre **tipo de día**, **tipo de semana** y **carga estimada**.

Este enfoque convierte el modelo en una herramienta útil para:
- **Planificar sesiones futuras**, ajustando cargas según contexto.  
- **Simular microciclos** y visualizar cómo respondería un jugador ante distintas distribuciones de entrenamiento.  
- Servir como base para futuros modelos de **recomendación automática de cargas** o **detección de riesgo de fatiga**.

---

## 💾 Archivos generados
Los artefactos finales guardados en `/src/chivas_ml/ml/registry/` fueron:

- `model_carga_diaria_rf_tendencias.pkl`  
- `scaler_carga_diaria.pkl`  
- `label_tipo_semana.pkl`  
- `label_tipo_dia.pkl`

---

## 🧠 Próximos pasos

1. Entrenar modelos similares para otras métricas (`Player_Load`, `HSR_abs_m`, `Acc_3`, `Dec_3`).  
2. Validar coherencia cruzada entre métricas (ej.: Player Load vs Distancia_total).  
3. Integrar el modelo al dashboard de Power BI para **simular cargas diarias planificadas**.  
4. Ajustar hiperparámetros y evaluar posibles mejoras con **XGBoost Regressor** o **LightGBM**.

---

📅 *Versión del modelo:* `v1.2 – Tendencias Fisiológicas (Random Forest)`  
👨‍💻 *Desarrollado por:* Nicolás Di Bartolo  
🧩 *Proyecto:* Chivas-ML — Machine Learning aplicado al entrenamiento deportivo
